In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('pgf')
'pgf'
import numpy as np
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os
import pickle
from models.gan.gan import generator, discriminator, StockTimeGan
from utils.log import prepare_logger
from pathlib import Path
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime

plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    "figure.figsize": (30, 10), 
    "figure.dpi": 600
})


logger = prepare_logger("INFO")
# %load_ext blackcellmagic

# %%black for formatting


In [2]:
TICKER = "EA"
MODEL_VERSION = 0.6

In [3]:
load_path = Path(os.path.abspath("")).parents[0] / "data" / "scaled_data"
models_path = Path(os.path.abspath("")).parents[0] / "models" / "gan" / "versions"

with open(load_path / f"data_{TICKER}.pickle", "rb") as handle:
    data = pickle.load(handle)

with open(load_path / f"scalers_{TICKER}.pickle", "rb") as handle:
    scalers = pickle.load(handle)

with open(models_path / f"model_{MODEL_VERSION}_{TICKER}.pickle", "rb") as handle:
    model_data = pickle.load(handle)


In [4]:
test_preds = scalers["y_scaler"].inverse_transform(model_data["test_preds"])
test_true = scalers["y_scaler"].inverse_transform(model_data["actual_values"])


In [5]:
test_true

array([[136.45770739],
       [139.22439748],
       [141.76234143],
       [141.16325753],
       [140.64041399],
       [140.22650314],
       [139.4858109 ],
       [139.87794356],
       [141.40289276],
       [140.98697779],
       [142.18000167],
       [140.75712938],
       [140.23175562],
       [138.55713983],
       [137.14520674],
       [137.71435398],
       [136.97007937],
       [138.30539633],
       [135.41586155],
       [135.03278087],
       [135.9521745 ],
       [134.96711228],
       [134.92333044],
       [132.23080978],
       [133.7521933 ],
       [136.98102692],
       [138.91833376],
       [137.13425919],
       [139.50936776],
       [136.57605949],
       [137.40788944],
       [137.3312733 ],
       [136.18203126],
       [137.67058049],
       [138.32729142],
       [140.85563226],
       [139.5422104 ],
       [140.77901612],
       [140.85563226],
       [141.65463627],
       [141.59990687],
       [141.59990687],
       [140.72428673],
       [139

In [6]:
test_true[0][0]

136.45770739287113

In [7]:
test_true[-1]

array([99.29666853])

In [8]:
amount = round(1000 / test_true[0][0])
rest = 1000 - 67 * test_true[0][0]
amount

7

In [9]:
67 * test_true[-1][0] + rest

-1489.7896036567327

In [10]:
# plt.plot(data["Y_preds_real_list_train"])
# plt.show()

In [11]:
# plt.plot(data["Y_preds_real_list_test"])
# plt.show()

In [12]:
# full_y = list(scalers["y_scaler"].inverse_transform(data["Y_preds_real_list_train"]))
# full_y.extend((list(scalers["y_scaler"].inverse_transform(data["Y_preds_real_list_test"]))))

# plt.plot(full_y)
# plt.axvline(535, label='Train/Test boundary')
# plt.title('EA Close price')
# plt.legend()
# plt.show()

In [13]:
data["Y_preds_real_list_train"].shape

(535, 1)

In [14]:
data["X_list_train"].shape

(535, 30, 19)

In [15]:
data["Y_preds_real_list_test"].shape

(178, 1)

In [16]:
mean_absolute_error(y_true=test_true, y_pred=test_preds)


1.342706735507938

In [17]:
np.sqrt(mean_squared_error(y_true=test_true, y_pred=test_preds))


1.8956227589829537

In [18]:
df = pd.DataFrame(
    {
        "predictions": test_preds.reshape(
            test_preds.shape[0],
        ),
        "actual_values": test_true.reshape(
            test_true.shape[0],
        ),
    }
)


In [19]:
stock_path = Path(os.path.abspath("")).parents[0] / "data" / "stock_data" / f'{TICKER}.csv'
stock_df = pd.read_csv(stock_path)

In [20]:
stock_df["Date"] = pd.to_datetime(stock_df["Date"])
dates = (
    stock_df[
        stock_df["Date"]
        >= datetime.datetime(year=2019, month=1, day=1) + datetime.timedelta(days=30)
    ]
    .tail(178)["Date"]
    .values
)


In [21]:
pd.DataFrame({"dates": dates}).to_csv(
    Path(os.path.abspath("")).parents[0] / "data" / "test_dates.csv", index=False
)


In [22]:
fig_save_path = Path(os.path.abspath("")).parents[0] / "paper"

In [23]:
fig_save_path = Path(os.path.abspath("")).parents[0] / "paper"
plt.plot(
    dates,
    test_preds.reshape(
        test_preds.shape[0],
    ),
    color="blue",
    label="Predicted price",
)
plt.plot(
    dates,
    test_true.reshape(
        test_true.shape[0],
    ),
    color="red",
    label="Actual price",
)
plt.title(f"{TICKER} actual price vs predicted price", size=25)
plt.xlabel("Date", size=25)
plt.ylabel("Valuation", size=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=25)
plt.show()
plt.savefig(fig_save_path / f'{TICKER}_prices.pgf')


C:\Users\kuba1\AppData\Local\Temp\ipykernel_41624\2075643222.py:24: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [24]:
np.median(test_true)


126.51740882298782

In [25]:
np.std(test_true)


12.51482057626007

In [26]:
results = {"1000_epochs_no_sentiment": {"mea": 1.2186, "rmse": 1.7288}}
plt.clf()

In [27]:
plt.plot(
    model_data['train_history']['gen_loss'],
    color="blue",
    label="Generator loss",
)
plt.plot(
    model_data['train_history']['disc_loss'],
    color="red",
    label="Discriminator Loss",
)
plt.title(f"{TICKER} generator and discriminator loss", size=25)
plt.xlabel("Epoch", size=25)
plt.ylabel("Loss", size=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=25)
plt.show()
plt.savefig(fig_save_path / f'{TICKER}_loss.pgf')

C:\Users\kuba1\AppData\Local\Temp\ipykernel_41624\540569605.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [28]:
model_data['train_history'].keys()

dict_keys(['gen_loss', 'disc_loss', 'real_y', 'pred_y'])

O czym napisac: 
- Spotkanie 
- Problemy ze stabilizacja modelu 
- Wywalenie dense, dodawanie i wywalanie sentymentu 
- Co z innymi modelami 
- Co jeszcze potrzeba 
- Wykresy? 
- Czy inne firmy 